### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__農林水産省 作況調査（果樹）__  
  URL:  
    ・https://www.maff.go.jp/j/tokei/kouhyou/sakumotu/sakkyou_kazyu/index.html  
  DATA:  
    ・f006-01-025.xls  
    ・f006-02-025.xls  
    ・f006-15-057.xls  
    ・f006-16-058.xls  
    ・f006-17-058.xls  
    ・f006-18-055.xls  
    ・f006-19-024.xls  
    ・f006-20-024.xls  
    ・f006-21-024.xls  
    ・f006-22-025.xls  
    ・f006-23-026.xls  
    ・f006-24-026.xls  
    ・f006-25-025.xls  
    ・f006-26-025.xls  
    ・f006-27-025.xls  
    ・f006-28-025.xls  
    ・f006-29-025.xls  
    ・f006-30-025.xls  
  データの取得方法:  
    「確報（統計表一覧）」の「調査結果データ〔Excel：e-Stat〕」にある、「令和2年産」〜「平成15年産」の各リンク先ページに移行します。  
    表番号３「〇〇〇年産都道府県別の結果樹面積・10a当たり収量・収穫量・出荷量」「ぶどう」（あれば「計」）行の「EXCEL」ボタンを押します。
    
---

__e-Stat 家計調査（二人以上の世帯）　品目別都道府県庁所在市及び政令指定都市（※）ランキング__  
__（2018年（平成30年）～2020年（令和２年）平均）__  
  URL:  
    ・https://www.stat.go.jp/data/kakei/5.html  
  DATA:  
    ・rank07.xlsx  
  データの取得方法:  
    「果物（エクセル：33KB）」のリンクをクリックします。
    
---

__総務省 全国地方公共団体コード__  
  URL:  
    ・https://www.soumu.go.jp/denshijiti/code.html   
  DATA:  
    ・000730858.xlsx  
  データの取得方法:  
    「コード一覧表」の「都道府県コード及び市区町村コード」にある「Excelファイル」のリンクをクリックします。

In [ ]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import xlrd

### 1. データの概要を掴む

今回は沢山のデータファイルがありますが、データの内容を把握するため、まずは令和２年産のぶどうのデータをロードします。

In [ ]:
grape_02_df = pd.read_excel('data/f006-02-025.xls')

grape_02_df.head(15)

表の位置がずれているようなので、Excelファイルを確認しながら`index_col`と`header`を指定して再読み込み。

In [ ]:
grape_02_df = pd.read_excel('data/f006-02-025.xls', index_col=1, header=7)

grape_02_df.head(15)

次に列、行の修正・整理をします。

列：  
・インデックス名を「都道府県」に修正。  
・コラム名を正しく指定できるよう「10ａ当たり\n収 量」のようなコラム名に含まれるホワイトスペースをすべて削除。  
・最初の２つの空の列を削除。  
・最後の「対前年産比」の４列も今回必要ないので削除。

行：  
・今回は都道府県単位のデータのみ残します。

In [ ]:
# インデックス名を修正
grape_02_df.index.name = '都道府県'

# コラム名にあるホワイトスペースをすべて削除
grape_02_df.columns = list(map(lambda x: "".join(x.split()), grape_02_df.columns.values))

# 不要な列を削除
grape_02_df = grape_02_df.loc[:, '結果樹面積':'出荷量']

# 不要な行を削除
# 最後の北海道〜最後の宮崎、鹿児島、沖縄のいずれかでスライス
last_hokkaido_index = np.where(grape_02_df.index == '北海道')[0][-1]
last_pref_index = np.where(np.isin(grape_02_df.index, ('大分', '宮崎', '鹿児島', '沖縄')))[0][-1]
grape_02_df = grape_02_df.iloc[last_hokkaido_index:last_pref_index+1]

grape_02_df

統計量を確認。

In [ ]:
grape_02_df.describe()

統計量が計算されません。

おそらくデータが数値として認識されていないようなので、データ型を確認。

In [ ]:
grape_02_df.info()

すべて`int64`型に変換してもう一度`info()`で確認します。

In [ ]:
grape_02_df = grape_02_df.astype(np.int64)

grape_02_df.info()

`int64`型に変換されたので、もう一度統計量を確認します。

In [ ]:
grape_02_df.describe()

４つの属性を、４つの棒グラフ表示してみましょう。

作業しやすいように`都道府県`をインデックスから外します。

In [ ]:
grape_02_df.reset_index(inplace=True)

grape_02_df.head()

グラフを描画します。

どんなことが読み取れますか？

In [ ]:
sns.set(font=['MS Gothic', 'Hiragino Sans', 'TakaoPGothic']) # 日本語フォントは設定が必要

fig, axes = plt.subplots(2, 2, sharex=True, figsize=(20,10))
sns.barplot(x='都道府県', y='結果樹面積', data=grape_02_df, ax=axes[0,0])

sns.barplot(x='都道府県', y='10ａ当たり収量', data=grape_02_df, ax=axes[0,1])

sns.barplot(x='都道府県', y='収穫量', data=grape_02_df, ax=axes[1,0])
axes[1,0].tick_params('x', labelrotation=90)

sns.barplot(x='都道府県', y='出荷量', data=grape_02_df, ax=axes[1,1])
axes[1,1].tick_params('x', labelrotation=90)

### 2. データフレームを縦に連結する

複数のデータフレーム同士を縦に連結します。

まずは先ほどと同じように、令和元年産のぶどうのデータをロードします。

In [ ]:
grape_01_df = pd.read_excel('data/f006-01-025.xls', index_col=1, header=7)

grape_01_df.head(15)

データがまたずれてしまっています。  
元のExcelファイルを確認してみると、先ほどと少し表の位置がずれているようです。

どのファイルからも正しい位置が分かるよう`xlrd`を使ってヘッダ行、インデックス列を検出してみます。  
（`openpyxl`は`.xls`をサポートしていません。）

検出条件  
・ヘッダ行： 「結果樹面積」を含む行。ただし「結果樹」の後にホワイトスペースが入っているケース有。  
・インデックス列： 「北海道」を含む列。

In [ ]:
header_row_index = None
index_col_index = None

with xlrd.open_workbook('data/f006-01-025.xls') as workbook:
    sheet = workbook.sheets()[0]

    max_row_index = 20
    max_col_index = 5
    for row_index in range(0, max_row_index + 1):
        for col_index in range(0, max_col_index + 1):
            cell_val = sheet.cell(row_index, col_index).value
            if not header_row_index and re.match('^結果樹', cell_val): 
                header_row_index = row_index
            elif not index_col_index and cell_val == '北海道':
                index_col_index = col_index
            
            if header_row_index is not None and index_col_index is not None:            
                break
        else:
            # ややトリッキーだが、breakでない場合はここを通る
            # 外側のforループを継続
            continue
        
        break # 外側のforループを出る
        
    print(f"ヘッダ行インデックス: {header_row_index}")
    print(f"インデックス列インデックス: {index_col_index}")

ヘッダ行のインデックスとインデックス列インデックスが分かったので、もう一度ロードし直します。

In [ ]:
grape_01_df = pd.read_excel('data/f006-01-025.xls', index_col=2, header=7)

grape_01_df

次に先ほどと同じ行・列の修正・整理から`都道府県`をインデックスから外すまでのセットアップ処理をおこなう関数を作成します。  
Excelファイルごとに若干違いがあるので、ファイルをいくつか確認しながら違いを吸収するようにします。

なお、関数では以下の処理を追加します。  
・`年度`列を追加し、データの年度を西暦で登録。  
・`ソース`列を追加し、データに問題があった時に確認できるようにデータ取得元のファイルのパスを登録する。  
・途中に空行が含まれるケースがあるため、空行を削除する処理を含める。

In [ ]:
def setup_grape_df(grape_df, year, path):
    """ぶどうの作況調査をロードしたデータフレームのセットアップ。

    Args:
        grape_df: ぶどうの作況調査をロードしたデータフレーム。
        year: データの年度（西暦）。
        
    Returns:
        DataFrame: セットアップ後のデータフレーム。
    """
    
    # インデックス名を修正
    grape_df.index.name = '都道府県'

    # コラム名にあるホワイトスペースをすべて削除
    grape_df.columns = list(map(lambda x: "".join(x.split()), grape_df.columns.values))

    # 不要な列を削除
    grape_df = grape_df.loc[:, '結果樹面積':'出荷量']

    # 不要な行を削除
    # 最後の北海道〜最後の宮崎、鹿児島、沖縄のいずれかでスライス
    last_hokkaido_index = np.where(grape_df.index == '北海道')[0][-1]
    last_pref_index = np.where(np.isin(grape_df.index, ('大分', '宮崎', '鹿児島', '沖縄')))[0][-1]
    grape_df = grape_df.iloc[last_hokkaido_index:last_pref_index+1]

    # 空行を削除
    grape_df.dropna(how='all', inplace=True)
        
    # データ型を変換
    grape_df = grape_df.astype(np.int64)
    
    # 作業しやすいようにをインデックから外す
    grape_df.reset_index(inplace=True)
    
    # 年度列を追加し、年度を登録
    grape_df['年度'] = pd.Timestamp(year=year, month=1, day=1)

    # ソース列を追加し、ファイル名を登録
    grape_df['ソース'] = path
    
    return grape_df

関数を実行してデータフレームをセットアップします。

In [ ]:
grape_01_df = setup_grape_df(grape_01_df, 2019, 'data/f006-01-025.xls') # 令和元年は2019年

grape_01_df

では令和2年のデータと令和元年のデータを`concat()`で連結してみましょう。

令和2年のデータには`年度`列がないため、`年度`列を追加してから令和元年のデータを連結します。

In [ ]:
grape_02_df['年度'] = pd.Timestamp(year=2020, month=1, day=1) # 令和2年は2020年
grape_reiwa_df = pd.concat([grape_01_df, grape_02_df])

print(f"令和元年のデータ件数: {len(grape_01_df)}")
print(f"令和2年のデータ件数: {len(grape_02_df)}")
print(f"連結後のデータ件数: {len(grape_reiwa_df)}")

連結後のデータを確認。

In [ ]:
grape_reiwa_df

インデックスが連結前のままになっているので`reset_index(drop=True)`を実行してインデックスを振り直します。

または事前に`concat()`の引数に`ignore_index=True`と指定することでもインデックスを振り直すことができます。

In [ ]:
grape_reiwa_df = grape_reiwa_df.reset_index(drop=True)

grape_reiwa_df

問題なく連結できることが確認できましたので、繰り返し処理で平成15年から令和2年までの全データを連結します。

In [ ]:
grape_df_list = []
excel_files = glob.glob(os.path.join('data', "f006-*.xls"))
for file in excel_files:
            
    year_in_era = int(re.search('f006-(\d{2})-\d{3}.xls$', file).group(1))
    year = year_in_era + (1988 if year_in_era >= 15 else 2018) # 平成15〜30年は2003〜2018年、令和元年は2019年
    
    header_row_index = None
    index_col_index = None

    with xlrd.open_workbook(file) as workbook:
        sheet = workbook.sheets()[0]

        max_row_index = 20
        max_col_index = 5
        for row_index in range(0, max_row_index + 1):
            for col_index in range(0, max_col_index + 1):
                cell_val = sheet.cell(row_index, col_index).value
                
                if not header_row_index and re.match('^結果樹', cell_val): 
                    header_row_index = row_index
                elif not index_col_index and cell_val == '北海道':
                    index_col_index = col_index
                
                if header_row_index is not None and index_col_index is not None:
                    break
            else:
                # ややトリッキーだが、breakでない場合はここを通る
                # 外側のforループを継続
                continue
        
            break # 外側のforループを出る

    if header_row_index is None or index_col_index is None:
        print(header_row_index, index_col_index)
        raise ValueError(f"ファイル'{file}'の表が検出できません。")

        
    grape_df = pd.read_excel(file, index_col=index_col_index, header=header_row_index)

    grape_df = setup_grape_df(grape_df, year, file)

    grape_df_list.append(grape_df)
    
grape_df = pd.concat(grape_df_list, ignore_index=True)

print(f"データ件数: {len(grape_df)}")

中身を確認。

In [ ]:
grape_df.head()

`10ａ当たり収量`と`10a当たり収量`の表記のずれがあって、連結に失敗しています。  
本来は関数で処理すべきですが、このデータフレームを直接修正してしまいます。

まず`10ａ当たり収量`列の値が欠損している場合、`10a当たり収量`列の値を登録します。

In [ ]:
grape_df['10ａ当たり収量'] = grape_df.apply(
    lambda x: x['10ａ当たり収量'] if not np.isnan(x['10ａ当たり収量']) else x['10a当たり収量'], axis=1)

次に`10ａ当たり収量`列で欠損がないかを確認します。

In [ ]:
grape_df['10ａ当たり収量'].isna().sum()

問題なければ`10a当たり収量`列を削除します。

In [ ]:
grape_df.drop(columns=['10a当たり収量'], inplace=True)

grape_df.head()

今度は行に問題がないか、`都道府県`列の値を確認。

In [ ]:
grape_df['都道府県'].value_counts()

都道府県でないものが１つずつ混じってしまっています。

データを確認します。

In [ ]:
unwanted_prefs = grape_df['都道府県'].value_counts()[grape_df['都道府県'].value_counts() == 1].index.values

grape_df[grape_df['都道府県'].isin(unwanted_prefs)]

どれも同じファイルからのデータのようで、そのファイルの中身を確認すると順番に問題があることが分かります。

削除して問題なさそうなので削除してしまいます。

In [ ]:
grape_df = grape_df[~grape_df['都道府県'].isin(unwanted_prefs)]

grape_df.head()

もう一度`都道府県`列の値を確認。

In [ ]:
grape_df['都道府県'].value_counts()

統計量を確認。

In [ ]:
grape_df.describe()

先ほどと同じように棒グラフで表示してみます。  
ただし複数年のデータがあるので、都道府県ごとの平均値を使います。

都道府県ごとの平均のデータフレームを用意します。

In [ ]:
grape_groupby_pref = grape_df.groupby('都道府県').mean().reset_index()

grape_groupby_pref

都道府県の順序が狂ってしまいました。

以下の手順で順序を並べ替えます。  
1.`都道府県`列のデータをカテゴリ型に変換。  
2.`都道府県`列のカテゴリのリストを令和2年産の`都道府県`列の順序に変更。

In [ ]:
grape_groupby_pref['都道府県'] = grape_groupby_pref['都道府県'].astype('category')

grape_groupby_pref['都道府県'] = grape_groupby_pref['都道府県'].cat.set_categories(grape_02_df['都道府県'])

grape_groupby_pref['都道府県'].cat.categories

グラフを描画します。

どんなことが読み取れますか？

In [ ]:
### チャレンジしてみましょう！！ ###

今度は結果樹面積の平均が500を超えている都道府県について、各数値の推移を折れ線グラフで表示してみます。

まず、結果樹面積の平均が500を超えている都道府県のデータを抽出したデータフレームを作成します。

In [ ]:
top_pref = grape_groupby_pref[grape_groupby_pref['結果樹面積'] > 500]['都道府県']

grape_top_pref_df = grape_df[grape_df['都道府県'].isin(top_pref)]

grape_top_pref_df

グラフを表示します。

どんなことが読み取れますか？

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20,16))

sns.lineplot(x='年度', y='結果樹面積', hue='都道府県', data=grape_top_pref_df, ax=axes[0,0])
axes[0, 0].get_legend().remove()

sns.lineplot(x='年度', y='10ａ当たり収量', hue='都道府県', data=grape_top_pref_df, ax=axes[0,1])
axes[0, 1].legend(loc="upper left", bbox_to_anchor=(1,1))

sns.lineplot(x='年度', y='収穫量', hue='都道府県', data=grape_top_pref_df, ax=axes[1,0])
axes[1, 0].get_legend().remove()

sns.lineplot(x='年度', y='出荷量', hue='都道府県', data=grape_top_pref_df, ax=axes[1,1])
axes[1, 1].legend(loc="upper left", bbox_to_anchor=(1,1))

### 3. 過去・未来のデータとの結合

ここでは移動平均を取得する目的で、過去および未来のデータとの結合をおこないます。

先ほどの`10a当たり収量`の折れ線グラフでは線の上下が激しすぎて、トレンドが把握しにくい状態でした。  
そこで`10a当たり収量`と`収穫量`の3年間の単純移動平均を求めてグラフに表示してみましょう。

グラフ表示に必要なデータフレームの作成手順は以下のようになります。
1. データフレームを年度の昇順にソートする。
2. `rolling()`を使って各行ごとに自身と前後の行の平均、つまり3年間の単純移動平均を計算したデータフレームを作成。
3. 元のデータフレームに単純移動平均のデータフレームを結合。

まずデータを年度の昇順にソートします。

普通にソートしても良いのですが、練習も兼ねて今回必要なグループ単位ごとでソートするようにします。  
少々複雑になりますがデータのサイズが大きい場合、そうすることである程度の処理速度の向上が見込めます。

In [ ]:
grape_top_pref_grouped_and_sorted = grape_top_pref_df.groupby('都道府県') \
                                    .apply(lambda g: g.sort_values('年度', ascending=True))

grape_top_pref_grouped_and_sorted

出来上がったデータフレームにはインデックスとコラムの両方に`都道府県`が含まれていますので、インデックスの方の`都道府県`を削除します。  
ただし、インデックスは`都道府県`と数値のインデックスの２つありますので、`都道府県`のみ削除します。

また、今回必要な３つの列`都道府県`、 `10ａ当たり収量`、 `収穫量`以外の列を削除します。

In [ ]:
grape_top_pref_grouped_and_sorted = grape_top_pref_grouped_and_sorted.reset_index(level='都道府県', drop=True)

grape_top_pref_grouped_and_sorted = grape_top_pref_grouped_and_sorted[['都道府県', '10ａ当たり収量', '収穫量']]

grape_top_pref_grouped_and_sorted

次に`groupby()`で都道府県でグループ化した`DataFrameGroupBy`オブジェクトを作成後、`rolling()`を実行します。  

自身の行を真ん中として前後１行ずつの3行の平均を求めたいので、`rolling()`の引数に`window=3`、`center=True`を指定し、`mean()`を実行します。

In [ ]:
grape_top_pref_moving_average_df = grape_top_pref_grouped_and_sorted \
                                                    .groupby('都道府県') \
                                                    .rolling(window=3, center=True) \
                                                    .mean()

grape_top_pref_moving_average_df

出来上がった`10ａ当たり収量`と`収穫量`の3年移動平均のデータフレームを元のデータフレームと結合したいので、２つのインデックスのうち`都道府県`の方を削除します。

In [ ]:
grape_top_pref_moving_average_df.reset_index(level='都道府県', drop=True, inplace=True)

grape_top_pref_moving_average_df

`join()`で２つデータフレームをインデックスの値をキーにして結合します。

両方に同じコラム名が存在するため、`join()`の引数に`rsuffix='_移動平均_3年'`として名前の衝突を避けるようにします。

In [ ]:
grape_top_pref_with_moving_average_df = grape_top_pref_df \
                                        .join(grape_top_pref_moving_average_df, rsuffix='_移動平均_3年')

grape_top_pref_with_moving_average_df

念のため、北海道のデータを`年度`でソートし、移動平均が正しく計算されているか確認します。

In [ ]:
grape_top_pref_with_moving_average_df.query('都道府県 == "北海道"').sort_values('年度')

それでは、`10ａ当たり収量`、`収穫量`それぞれの年ごとの値と移動平均の４つの折れ線グラフを表示します。

どんなことが読み取れますか？

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20,16))

sns.lineplot(x='年度', y='10ａ当たり収量', hue='都道府県', data=grape_top_pref_with_moving_average_df, ax=axes[0,0])
axes[0,0].get_legend().remove()

sns.lineplot(x='年度', y='10ａ当たり収量_移動平均_3年', hue='都道府県', data=grape_top_pref_with_moving_average_df,
             ax=axes[0,1])
axes[0,1].legend(loc="upper left", bbox_to_anchor=(1,1))


sns.lineplot(x='年度', y='収穫量', hue='都道府県', data=grape_top_pref_with_moving_average_df, ax=axes[1,0])
axes[1,0].get_legend().remove()

sns.lineplot(x='年度', y='収穫量_移動平均_3年', hue='都道府県', data=grape_top_pref_with_moving_average_df, ax=axes[1,1])
axes[1,1].legend(loc="upper left", bbox_to_anchor=(1,1))

### 4. キーを使ったデータの結合

２つのデータフレームをキーを指定して結合します。

ここからは各都道府県の収穫量と消費量の関係の分析をおこないます。

家計調査のぶどう購入数量データをロードします。  
Excelファイルを確認しながら、`header`と`index_col`を指定します。

In [ ]:
grape_consumption_df = pd.read_excel('data/rank07.xlsx', header=2, index_col=0)

grape_consumption_df.head()

ぶどうの列を確認。

金額列は`ぶどう`のまま、数量列は`ぶどう.1`という列名に変更されています。

In [ ]:
grape_consumption_df.filter(regex='^ぶどう', axis=1).head()

今回欲しいのは数量なので、`ぶどう.1`列と右隣の列だけを残して削除します。

In [ ]:
second_grape_column_index = np.where(grape_consumption_df.columns == 'ぶどう.1')[0][0]

grape_consumption_df = grape_consumption_df.iloc[:,second_grape_column_index:second_grape_column_index+2]

grape_consumption_df.head()

列名をそれぞれ`都市名`, `数量`に変更し、不必要な最初の２行も削除します。

In [ ]:
grape_consumption_df.columns = ['都市名', '購入数量']

grape_consumption_df = grape_consumption_df.iloc[2:]

grape_consumption_df.head()

データ数を確認します。

都道府県数以上のデータ数がありますが、家計調査が都道府県庁所在市と政令指定都市５つ（川崎市，相模原市，浜松市，堺市及び北九州市）を対象にしているためです。

In [ ]:
len(grape_consumption_df)

どんな都市名があるか確認。

In [ ]:
grape_consumption_df['都市名'].unique()

各都道府県の収穫量と各都市での購入数量を結びつけられるよう、どの都市がどの都道府県にあるのかを識別する列が必要です。  
そのため、今回は全国地方公共団体コードのデータを利用します。

まずデータをロードします。

In [ ]:
pref_df = pd.read_excel('data/000730858.xlsx')

pref_df.head()

データを加工します。

1.コラム名にホワイトスペースが混じっていて指定しにくいので、すべてのホワイトスペースを除去。  
2.`都道府県名（漢字）`、 `市区町村名（漢字）`以外の列は必要ないので削除。  
3.`都道府県名（漢字）`、 `市区町村名（漢字）`をそれぞれ`都道府県名`、 `市区町村名`に変更。

In [ ]:
pref_df.columns = list(map(lambda x: "".join(x.split()), pref_df.columns.values))

pref_df = pref_df[['都道府県名（漢字）', '市区町村名（漢字）']]

pref_df.columns = ['都道府県名', '市区町村名']

pref_df.head()

それではぶどう購入数量、都道府県名・市区町村名マッピングの２つのデータフレームを各データフレームのキーを指定しつつ結合します。

ぶどう購入数量のデータが全て残るよう、左結合にします。

In [ ]:
grape_consumption_with_pref_df = pd.merge(grape_consumption_df, pref_df, how='left',
                                          left_on="都市名", right_on="市区町村名")

grape_consumption_with_pref_df.head()

新たに生成したデータフレームの`都道府県名`の列に欠損がないか確認。

In [ ]:
grape_consumption_with_pref_df['都道府県名'].isna().sum()

どのデータで欠損があるのか確認します。

In [ ]:
grape_consumption_with_pref_df[grape_consumption_with_pref_df['都道府県名'].isna()]

これは仕方ないので、欠損値を直接修正します。

In [ ]:
grape_consumption_with_pref_df.loc[grape_consumption_with_pref_df['都市名'] == '東京都区部', '都道府県名'] \
    = '東京都'
grape_consumption_with_pref_df.loc[grape_consumption_with_pref_df['都市名'] == '東京都区部', '市区町村名'] \
    = '東京都区部'

grape_consumption_with_pref_df[grape_consumption_with_pref_df['都市名'] == '東京都区部']

今度は`都道府県名`の重複を`value_counts()`で確認。

すべての都道府県が揃っているかも確認します。

In [ ]:
grape_consumption_with_pref_df['都道府県名'].value_counts()

`都道府県名`の重複を解消するため、都道府県名でグループ化します。

重複した都道府県は数値を平均し、`都道府県名`をインデックスから外しましょう。

In [ ]:
grape_consumption_groupby_pref = grape_consumption_with_pref_df.groupby('都道府県名').mean().reset_index()

grape_consumption_groupby_pref.head()

加工したデータを棒グラフで表示してみましょう。

`都道府県名`列の順序が狂ってしまっているので、先ほどと同じにカテゴリ型に変換して順序を整えます。

In [ ]:
# 都道府県名の順序を全国地方公共団体コードのデータから抽出
pref_order = pref_df[pref_df['市区町村名'].isna()]['都道府県名'].tolist()

grape_consumption_groupby_pref['都道府県名'] = grape_consumption_groupby_pref['都道府県名'].astype('category')

grape_consumption_groupby_pref['都道府県名'] = grape_consumption_groupby_pref['都道府県名'] \
                                                .cat.set_categories(pref_order)

grape_consumption_groupby_pref['都道府県名'].cat.categories

グラフを表示してみましょう。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

sns.barplot(x='都道府県名', y='購入数量', data=grape_consumption_groupby_pref)
ax.tick_params('x', labelrotation=90)

グループ分けしたぶどう購入数量と結合するための、ブドウ収穫量のデータフレームを作成します。

ぶどう購入数量データは2018〜2020年の平均ですから、ブドウ収穫量もそれに合わせます。

In [ ]:
grape_yield_df = grape_df.query('2018 <= 年度 <= 2020').groupby('都道府県').agg({'収穫量': 'mean'})

grape_yield_df.head()

都道府県でグループ分けしたぶどう購入数量のデータフレームは「〇〇都」「〇〇府」「〇〇県」となっていますので、末尾の文字を取り除きます。

In [ ]:
grape_consumption_groupby_pref['都道府県名'] = grape_consumption_groupby_pref['都道府県名'] \
                                              .str.extract('(.+)[都府県]$') \
                                              .fillna('北海道')

ぶどう収穫量、都道府県でグループ分けしたぶどう購入数量の２つのデータフレームを各データフレームのキーを指定しつつ結合します。

各データフレームにすべての都道府県のデータが１件ずつありますので内部結合を指定します。

In [ ]:
grape_yield_consumption_df = pd.merge(grape_yield_df, grape_consumption_groupby_pref, how='inner',
                                          left_on="都道府県", right_on="都道府県名")

grape_yield_consumption_df.head(10)

データを回帰直線付きの散布図で表示します。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

sns.regplot(x='収穫量', y='購入数量', data=grape_yield_consumption_df, ci=0)

# 各点にラベルを追加
for row_num in range(0, grape_yield_consumption_df.shape[0]):
    ax.text(grape_yield_consumption_df['収穫量'][row_num],
            grape_yield_consumption_df['購入数量'][row_num], 
            grape_yield_consumption_df['都道府県名'][row_num],
            horizontalalignment='left', 
            size='small',
            color='gray',
            weight='normal')